In [7]:
!pip install cma

  Using cached cma-3.3.0-py3-none-any.whl (260 kB)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [8]:
import cma
import pickle
import numpy as np
import time

load data from `prepare_Xy.ipynb`.

In [9]:
# load data from `hydrogen_update_XY.ipynb`.
X = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['x']
print("shape of X:", np.shape(X))

y = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['y']
print("shape of y:", np.shape(y))

nb_COFs = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['nb_COFs']
print("# COFs:", nb_COFs)

nb_runs = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['nb_runs']
print("# runs:", nb_runs)

nb_iterations = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['nb_iterations']
print("# iterations:", nb_iterations)

# feature names
features = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['features']
print('features: ', features)

shape of X: (98694, 7)
shape of y: (98694,)
# COFs: 98694
# runs: 100
# iterations: 250
features:  ['Density', 'GSA', 'VSA', 'VF', 'PV', 'LCD', 'PLD']


In [10]:
# so evol search is bounded by [0, 1]^12
assert np.max(X) <= 1.0
assert np.min(X) >= 0.0

In [11]:
# return id of COF closest to x_ask in feature space, which is not in the set of acquired COFs
def closest_COF(x_ask, ids_acquired):
    distances_to_xs = np.linalg.norm(X - x_ask, axis=1)
    assert np.size(distances_to_xs) == nb_COFs
    ids_sorted_by_closeness = np.argsort(distances_to_xs)
    for id_closest in ids_sorted_by_closeness:
        if not id_closest in ids_acquired:
            return id_closest

In [5]:
# two tests
assert closest_COF(X[3, :], []) == 3

ids = [i for i in range(nb_COFs) if not i == 3]
distances = np.linalg.norm(X - X[3, :], axis=1)
assert distances[closest_COF(X[3, :], [3])] == np.min(distances[ids])

In [12]:
def evol_run(nb_iterations):
    x_init = X[np.random.choice(np.arange((nb_COFs)), size=1, replace=False)[0], :] # random initial COF

    # initialize evolutionary algo
    ids_acquired = []
    es = cma.CMAEvolutionStrategy(x0=x_init, sigma0=0.5, inopts={'bounds': [0.0, 1.0]})

    while len(ids_acquired) < nb_iterations:
        print("ask/tell sesh")
        # ask for a set of COFs to evaluate.
        xs_ask = es.ask()

        # find IDs of these COFs (the closest unacquired COFs in feature space to them.)
        ids_ask = []
        for x_ask in xs_ask:
            assert np.min(x_ask) >= 0.0
            assert np.max(x_ask) <= 1.0
            id_ask = closest_COF(x_ask, ids_acquired)
            ids_ask.append(id_ask)
            # officially acquire this COF
            ids_acquired.append(id_ask)

        # tell the ES algo about these most recently acquired COFs.
        es.tell(X[ids_ask, :], -y[ids_ask]) # cuz we wanna minimize

        print("\t# acquired COFs:", len(ids_acquired))
        print("\t# max y value: ", np.max(y[ids_acquired]))
    assert np.size(np.unique(ids_acquired[:nb_iterations]) == nb_iterations)
    return ids_acquired[:nb_iterations]

In [13]:
es_res = dict()
es_res['ids_acquired'] = []
for r in range(nb_runs):
    print("\n\nRUN", r)
    ids_acquired = evol_run(nb_iterations)
    es_res['ids_acquired'].append(ids_acquired)
    
with open('es_results_hydrogen.pkl', 'wb') as file:
    pickle.dump(es_res, file)



RUN 0
(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 7 (seed=811463, Mon Mar 13 12:42:35 2023)
ask/tell sesh


/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/cma/evolution_strategy.py:1901: UserWarning: Sampling standard deviation i=0 at iteration 0 change by 0.6666666666666666 to stds[0]=0.3333333333333333
  warnings.warn("Sampling standard deviation i={} at iteration {}"
/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/cma/evolution_strategy.py:1901: UserWarning: Sampling standard deviation i=1 at iteration 0 change by 0.6666619047789115 to stds[1]=0.3333333333333333
  warnings.warn("Sampling standard deviation i={} at iteration {}"
/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/cma/evolution_strategy.py:1901: UserWarning: Sampling standard deviation i=2 at iteration 0 change by 0.6666571429251698 to stds[2]=0.3333333333333333
  warnings.warn("Sampling standard deviation i={} at iteration {}"
/home/santanu/anaconda3/envs/sage/lib/python3.8/site-packages/cma/evolution_strategy.py:1901: UserWarning: Sampling standard deviation i=3 at iteration 0 change

	# acquired COFs: 9
	# max y value:  10.27
ask/tell sesh
	# acquired COFs: 18
	# max y value:  13.07
ask/tell sesh
	# acquired COFs: 27
	# max y value:  22.4
ask/tell sesh
	# acquired COFs: 36
	# max y value:  22.4
ask/tell sesh
	# acquired COFs: 45
	# max y value:  22.6
ask/tell sesh
	# acquired COFs: 54
	# max y value:  28.77
ask/tell sesh
	# acquired COFs: 63
	# max y value:  29.94
ask/tell sesh
	# acquired COFs: 72
	# max y value:  35.75
ask/tell sesh
	# acquired COFs: 81
	# max y value:  40.46
ask/tell sesh
	# acquired COFs: 90
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 99
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 108
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 117
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 126
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 135
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 144
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 153
	# max y value:  45.71
ask/tell sesh
	# acquired 

	# acquired COFs: 216
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 225
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 234
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 243
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 252
	# max y value:  45.71


RUN 5
(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 7 (seed=712575, Mon Mar 13 12:42:53 2023)
ask/tell sesh
	# acquired COFs: 9
	# max y value:  22.93
ask/tell sesh
	# acquired COFs: 18
	# max y value:  23.58
ask/tell sesh
	# acquired COFs: 27
	# max y value:  24.37
ask/tell sesh
	# acquired COFs: 36
	# max y value:  37.83
ask/tell sesh
	# acquired COFs: 45
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 54
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 63
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 72
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 81
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 90
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 99
	# max 

	# acquired COFs: 144
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 153
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 162
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 171
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 180
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 189
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 198
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 207
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 216
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 225
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 234
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 243
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 252
	# max y value:  45.71


RUN 10
(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 7 (seed=838043, Mon Mar 13 12:43:09 2023)
ask/tell sesh
	# acquired COFs: 9
	# max y value:  9.25
ask/tell sesh
	# acquired COFs: 18
	# max y value:  21.28
ask/tell sesh
	# acquired COFs: 27

...

	# acquired COFs: 216
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 225
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 234
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 243
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 252
	# max y value:  45.71


RUN 81
(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 7 (seed=751976, Mon Mar 13 12:47:10 2023)
ask/tell sesh
	# acquired COFs: 9
	# max y value:  5.48
ask/tell sesh
	# acquired COFs: 18
	# max y value:  13.07
ask/tell sesh
	# acquired COFs: 27
	# max y value:  18.39
ask/tell sesh
	# acquired COFs: 36
	# max y value:  35.3
ask/tell sesh
	# acquired COFs: 45
	# max y value:  41.7
ask/tell sesh
	# acquired COFs: 54
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 63
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 72
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 81
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 90
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 99
	# max y 

	# acquired COFs: 144
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 153
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 162
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 171
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 180
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 189
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 198
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 207
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 216
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 225
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 234
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 243
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 252
	# max y value:  45.71


RUN 86
(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 7 (seed=777670, Mon Mar 13 12:47:27 2023)
ask/tell sesh
	# acquired COFs: 9
	# max y value:  12.73
ask/tell sesh
	# acquired COFs: 18
	# max y value:  30.7
ask/tell sesh
	# acquired COFs: 27

	# acquired COFs: 72
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 81
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 90
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 99
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 108
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 117
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 126
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 135
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 144
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 153
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 162
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 171
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 180
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 189
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 198
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 207
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 216
	# max y value:  45.71
ask/tell sesh
	

	# acquired COFs: 9
	# max y value:  28.15
ask/tell sesh
	# acquired COFs: 18
	# max y value:  32.75
ask/tell sesh
	# acquired COFs: 27
	# max y value:  32.75
ask/tell sesh
	# acquired COFs: 36
	# max y value:  32.75
ask/tell sesh
	# acquired COFs: 45
	# max y value:  37.56
ask/tell sesh
	# acquired COFs: 54
	# max y value:  38.62
ask/tell sesh
	# acquired COFs: 63
	# max y value:  42.15
ask/tell sesh
	# acquired COFs: 72
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 81
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 90
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 99
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 108
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 117
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 126
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 135
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 144
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 153
	# max y value:  45.71
ask/tell sesh
	# acquir

	# acquired COFs: 207
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 216
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 225
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 234
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 243
	# max y value:  45.71
ask/tell sesh
	# acquired COFs: 252
	# max y value:  45.71


In [14]:
with open('es_results_hydrogen.pkl', 'wb') as file:
    pickle.dump(es_res, file)

In [15]:
print(es_res)

{'ids_acquired': [[95715, 92096, 87761, 70727, 59285, 66067, 40211, 74109, 19590, 43242, 2859, 23587, 18367, 46801, 23975, 42309, 53761, 70256, 13313, 83821, 15869, 40212, 37527, 7378, 8413, 20846, 10026, 6222, 12924, 79386, 71087, 16382, 11704, 48279, 3397, 17582, 19604, 25446, 16480, 11048, 16, 19858, 12416, 11191, 14945, 20676, 14879, 4620, 4037, 13495, 13013, 3467, 28620, 24261, 28464, 27315, 25127, 3587, 31776, 24555, 12003, 25325, 11545, 29415, 21992, 28748, 33515, 31081, 30117, 22195, 27831, 30727, 18053, 23890, 31201, 25591, 33872, 31530, 34166, 32999, 28619, 29134, 35496, 33581, 35235, 34094, 32450, 36414, 35679, 26977, 34291, 34290, 17455, 31123, 31080, 35168, 33918, 33713, 32843, 27654, 31975, 35131, 32560, 30919, 30797, 25083, 24074, 26305, 17756, 33795, 29000, 25267, 32717, 32561, 27708, 32998, 28132, 32559, 32274, 32060, 34167, 33220, 29848, 32371, 35132, 22242, 32558, 34054, 31973, 34095, 34130, 34533, 31202, 30114, 32401, 32156, 31699, 30951, 32008, 31200, 31600, 32157,